### Importing the dependencies

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from keras import layers
from keras.models import Sequential 
from keras.layers import RNN, LSTM, GRU, Bidirectional, Embedding, Dense
from keras.datasets import imdb

### Loading the data

In [2]:
#Loading the data from the imdb dataset
max_freq = 10000 #only the top 10000 words are included.
(x_train,y_train) , (x_test, y_test) = imdb.load_data(num_words = max_freq)

In [3]:
print('---review---')
print(x_train[6])
print('---label---')
print(y_train[6])

---review---
[1, 6740, 365, 1234, 5, 1156, 354, 11, 14, 5327, 6638, 7, 1016, 2, 5940, 356, 44, 4, 1349, 500, 746, 5, 200, 4, 4132, 11, 2, 9363, 1117, 1831, 7485, 5, 4831, 26, 6, 2, 4183, 17, 369, 37, 215, 1345, 143, 2, 5, 1838, 8, 1974, 15, 36, 119, 257, 85, 52, 486, 9, 6, 2, 8564, 63, 271, 6, 196, 96, 949, 4121, 4, 2, 7, 4, 2212, 2436, 819, 63, 47, 77, 7175, 180, 6, 227, 11, 94, 2494, 2, 13, 423, 4, 168, 7, 4, 22, 5, 89, 665, 71, 270, 56, 5, 13, 197, 12, 161, 5390, 99, 76, 23, 2, 7, 419, 665, 40, 91, 85, 108, 7, 4, 2084, 5, 4773, 81, 55, 52, 1901]
---label---
1


### Pre-processing the data

#### Word-Index Mapping

In [4]:
word_idx = imdb.get_word_index()
# originally in the word_idx, the words are the keys and the values are the pairs, we want it the other way round

word_dict = {i:word for word,i in word_idx.items()}
print([word_dict[i] for i in x_train[0]])

['the', 'as', 'you', 'with', 'out', 'themselves', 'powerful', 'lets', 'loves', 'their', 'becomes', 'reaching', 'had', 'journalist', 'of', 'lot', 'from', 'anyone', 'to', 'have', 'after', 'out', 'atmosphere', 'never', 'more', 'room', 'and', 'it', 'so', 'heart', 'shows', 'to', 'years', 'of', 'every', 'never', 'going', 'and', 'help', 'moments', 'or', 'of', 'every', 'chest', 'visual', 'movie', 'except', 'her', 'was', 'several', 'of', 'enough', 'more', 'with', 'is', 'now', 'current', 'film', 'as', 'you', 'of', 'mine', 'potentially', 'unfortunately', 'of', 'you', 'than', 'him', 'that', 'with', 'out', 'themselves', 'her', 'get', 'for', 'was', 'camp', 'of', 'you', 'movie', 'sometimes', 'movie', 'that', 'with', 'scary', 'but', 'and', 'to', 'story', 'wonderful', 'that', 'in', 'seeing', 'in', 'character', 'to', 'of', '70s', 'musicians', 'with', 'heart', 'had', 'shadows', 'they', 'of', 'here', 'that', 'with', 'her', 'serious', 'to', 'have', 'does', 'when', 'from', 'why', 'what', 'have', 'critics', 

In [5]:
# Get the minimum and the maximum length of reviews
print("Max length of a review:: ", len(max((x_train+x_test), key=len)))
print("Min length of a review:: ", len(min((x_train+x_test), key=len)))

Max length of a review::  2494
Min length of a review::  7


#### Padding the inputs

In [6]:
#we can see the revies are of varoed lengths, but to feed our data into a RNN, we need all the reviews to be of the same size. We carry out padding to take care of this issue
max_len = 800
from keras.preprocessing import sequence
x_train = sequence.pad_sequences(x_train, maxlen=max_len)
x_test = sequence.pad_sequences(x_test, maxlen=max_len)

In [7]:
x_train.shape

(25000, 800)

### Creating the models

In [8]:
#Fixing Embedding size to 64
embed_len = 64

#### SimpleRNN model

In [9]:
def SimpleRNN_Model():
    model = Sequential(
        [
            layers.Input(shape=(max_len,)),
            layers.Embedding(input_dim=max_freq, output_dim=embed_len),
            layers.SimpleRNN(128, activation='tanh', return_sequences=False),
            Dense(1, activation='sigmoid')
        ],
        name="Simple_RNN"
    )
    model._name = "Simple_RNN"
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

#### GRU Model

In [10]:
def GRU_Model():
    model = Sequential(
        [
            
            layers.Input(shape= (max_len,)),
            layers.Embedding(input_dim=max_freq, output_dim=embed_len),
            layers.GRU(128,activation='tanh',return_sequences=False),
            Dense(1, activation='sigmoid')
        ],
        name="GRU"
    )
    model.compile(optimizer='adam' , loss = 'binary_crossentropy' , metrics=['accuracy'])
    return model

#### LSTM Model

In [11]:
def LSTM_Model():
    model = Sequential([
            layers.Input(shape= (max_len,)),
            layers.Embedding(input_dim=max_freq, output_dim=embed_len),
            layers.LSTM(128,activation='tanh',return_sequences=False),
            Dense(1, activation='sigmoid')
        ],
        name="LSTM"
    )
    model.compile(optimizer='adam' , loss = 'binary_crossentropy' , metrics=['accuracy'])
    return model



#### BiLSTM Model

In [12]:
def BiLSTM_Model():
    model = Sequential([
            layers.Input(shape= (max_len,)),
            layers.Embedding(input_dim=max_freq, output_dim=embed_len),
            layers.Bidirectional(LSTM(128,activation='tanh',return_sequences=False)),
            Dense(1, activation='sigmoid')
        ],
        name="BiLSTM"
    )
    model.compile(optimizer='adam' , loss = 'binary_crossentropy' , metrics=['accuracy'])
    return model


In [13]:
#creating the dictionary for all the models
models = {
    1  : SimpleRNN_Model(),
    2  : GRU_Model(),
    3  : LSTM_Model(),
    4  : BiLSTM_Model()    
}

#### Summary of each model

In [14]:
for i in range(1,5):
    print(models[i].summary())


Model: "Simple_RNN"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 800, 64)        │       640,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ (None, 128)            │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 664,833 (2.54 MB)

 Trainable params: 664,833 (2.54 MB)

 Non-trainable params: 0 (0.00 B)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ (None, 800, 64)        │       640,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 256)            │       197,632 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 837,889 (3.20 MB)

 Trainable params: 837,889 (3.20 MB)

 Non-trainable params: 0 (0.00 B)

None


In [15]:
models[1].name

'Simple_RNN'

### Training the model

In [16]:
def fit_model(model, X_train, Y_train, X_test, Y_test, Epochs=10, Batch_size=64):
    model.fit(X_train,Y_train,epochs = Epochs, batch_size= Batch_size,validation_split= 0.3)
    loss,accuracy = model.evaluate(X_test,Y_test,verbose= 2)
    return accuracy

In [21]:
accuracy = {} #storing the accuracy values for each of the model
for i in range(1,5):
    accuracy[models[i].name] = fit_model(models[i],x_train,y_train,x_test,y_test)


Epoch 1/10
274/274 ━━━━━━━━━━━━━━━━━━━━ 57s 209ms/step - accuracy: 0.5417 - loss: 0.6871 - val_accuracy: 0.6281 - val_loss: 0.6395
Epoch 2/10
274/274 ━━━━━━━━━━━━━━━━━━━━ 57s 209ms/step - accuracy: 0.7136 - loss: 0.5682 - val_accuracy: 0.7047 - val_loss: 0.5672
Epoch 3/10
274/274 ━━━━━━━━━━━━━━━━━━━━ 57s 209ms/step - accuracy: 0.8095 - loss: 0.4234 - val_accuracy: 0.6351 - val_loss: 0.6262
Epoch 4/10
274/274 ━━━━━━━━━━━━━━━━━━━━ 54s 199ms/step - accuracy: 0.7474 - loss: 0.5146 - val_accuracy: 0.6401 - val_loss: 0.6319
Epoch 5/10
274/274 ━━━━━━━━━━━━━━━━━━━━ 52s 191ms/step - accuracy: 0.7995 - loss: 0.4363 - val_accuracy: 0.7424 - val_loss: 0.5731
Epoch 6/10
274/274 ━━━━━━━━━━━━━━━━━━━━ 55s 200ms/step - accuracy: 0.8636 - loss: 0.3247 - val_accuracy: 0.7283 - val_loss: 0.6293
Epoch 7/10
274/274 ━━━━━━━━━━━━━━━━━━━━ 53s 194ms/step - accuracy: 0.8994 - loss: 0.2600 - val_accuracy: 0.7540 - val_loss: 0.5894
Epoch 8/10
274/274 ━━━━━━━━━━━━━━━━━━━━ 54s 195ms/step - accuracy: 0.9071 - loss: 0

In [22]:
#Accuracy of each of the models
accuracy

{'Simple_RNN': 0.7089599967002869,
 'GRU': 0.8568800091743469,
 'LSTM': 0.8393999934196472,
 'BiLSTM': 0.8501600027084351}